In [4]:
!pip install dash
!pip install dash_bootstrap_components
!pip install flask_caching

# Task
Change the `fig_wkts` plot from a bar chart to a line chart with markers.

In [5]:
import pandas as pd
import numpy as np
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
from flask_caching import Cache

# ===============================
# TEAM MAPPING
# ===============================
TEAM_INFO = {
    "Chennai Super Kings": ("CSK","#f1c40f"),
    "Mumbai Indians": ("MI","#045093"),
    "Royal Challengers Bangalore": ("RCB","#da291c"),
    "Kolkata Knight Riders": ("KKR","#3b0a45"),
    "Rajasthan Royals": ("RR","#ea1a8e"),
    "Sunrisers Hyderabad": ("SRH","#ff822a"),
    "Delhi Capitals": ("DC","#17449b"),
    "Punjab Kings": ("PBKS","#d71920"),
    "Gujarat Titans": ("GT","#041c2c"),
    "Lucknow Super Giants": ("LSG","#00b2a9"),
    "Gujarat Lions": ("GL","#f26522"),
    "Rising Pune Supergiant": ("RPS","#800080"),
    "Pune Warriors India": ("PWI","#00a3e0"),
    "Kochi Tuskers Kerala": ("KTK","#f26a21"),
    "Deccan Chargers": ("DCG","#1e4fa1")
}

def get_team_abbr(t):
    for full,(abbr,col) in TEAM_INFO.items():
        if isinstance(t,str) and full.lower()==t.lower():
            return abbr
    return "UNK"

def get_team_color(t):
    for full,(abbr,col) in TEAM_INFO.items():
        if isinstance(t,str) and full.lower()==t.lower():
            return col
    return "#888888"

# ===============================
# LOAD DATA
# ===============================
matches = pd.read_csv("matches.csv")
deliveries = pd.read_csv("deliveries.csv")

matches.rename(columns={"Id":"match_id","Season":"season","Date":"date",
                        "Winner":"winner","Player_Of_Match":"player_of_match"}, inplace=True)
matches["date"]=pd.to_datetime(matches["date"],errors="coerce")

deliveries.rename(columns={"Match_Id":"match_id","Inning":"inning","Over":"over","Ball":"ball",
                           "Batter":"batter","Bowler":"bowler","Batting_Team":"batting_team",
                           "Bowling_Team":"bowling_team","Batsman_Runs":"batsman_runs",
                           "Total_Runs":"total_runs","Is_Wicket":"is_wicket",
                           "Player_Dismissed":"player_dismissed","Extras_Type":"extras_type",
                           "Dismissal_Kind":"dismissal_kind"}, inplace=True)

deliveries["is_wicket"] = deliveries["player_dismissed"].notna().astype(int)

# ===============================
# DASH APP
# ===============================
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CYBORG]) # Changed theme to CYBORG for more vibrant colors
cache = Cache(app.server, config={"CACHE_TYPE":"simple"})

PLAYERS = sorted(pd.unique(pd.concat([deliveries.batter,deliveries.bowler]).dropna()))
SEASONS = sorted(matches.season.unique())

def valid_ball(df): return ~df["extras_type"].isin(["Wides","Noballs","Wide","No ball"])

# =============================== STAT HELPERS ===============================
def base(player):
    df = deliveries.merge(matches[["match_id","season","date"]],on="match_id",how="left").copy()
    df["is_bat"]=df["batter"].eq(player)
    df["is_bowl"]=df["bowler"].eq(player)
    return df

def bat_summary(p):
    d=base(p)[lambda x:x.is_bat]
    if d.empty: return {'inn':0,'r':0,'balls':0,'4':0,'6':0,'sr':0,'bp':0}
    d2=d[valid_ball(d)]
    r=d2.batsman_runs.sum(); b=len(d2)
    fours=(d2.batsman_runs==4).sum(); six=(d2.batsman_runs==6).sum()
    return {'inn':d.match_id.nunique(),'r':r,'balls':b,'4':fours,'6':six,
                'sr':round((r/b*100) if b else 0,1),
                'bp':round(((fours+six)/b*100) if b else 0,1)}

def bowl_summary(p):
    d=base(p)[lambda x:x.is_bowl]
    if d.empty: return dict(inn=0,w=0,eco=0,sr="-",dot=0)
    d2=d[valid_ball(d)]
    balls=len(d2); runs=d2.total_runs.sum()
    wk=d[(d.is_wicket==1)&(d.player_dismissed.notna())&
         (~d.dismissal_kind.isin(["Run Out","Obstructing The Field","Retired Hurt","Retired Out","Handled The Ball","Timed Out"]))]
    w=len(wk)
    eco=round(runs/(balls/6),2) if balls else 0
    sr=round(balls/w,1) if w else "-"
    dot=round((d2.total_runs.eq(0).sum()/balls*100),1) if balls else 0
    return dict(inn=d.match_id.nunique(),w=w,eco=eco,sr=sr,dot=dot)

# ===============================
# LAYOUT
# ===============================
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(dcc.Dropdown(id="p1",options=[{"label":p,"value":p} for p in PLAYERS],value=PLAYERS[0]), md=4),
        dbc.Col(dcc.RangeSlider(
            id="season",
            min=min(SEASONS),
            max=max(SEASONS),
            step=1,
            value=[min(SEASONS), max(SEASONS)], # Default to all seasons
            marks={str(s): str(s) for s in SEASONS}
        ), md=8),
    ]),
    html.Hr(),
    html.Div(id="batkpi"),
    dbc.Row([
        dbc.Col(dcc.Graph(id="runs"), md=6),
        dbc.Col(dcc.Graph(id="wkts"), md=6),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id="form"), md=6),
        dbc.Col(dcc.Graph(id="dismiss"), md=6),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id="batphase"), md=6),
        dbc.Col(dcc.Graph(id="bowlphase"), md=6),
    ]),
    html.Hr(),
    dbc.Row([ # Added new row for the animated plot
        dbc.Col(dcc.Graph(id="animated_runs"), md=12) # Full width for animated plot
    ]),
    # Removed the Comparison section as requested
    # html.H3("Comparison", className="text-light"),
    # dbc.Row([
    #     dbc.Col(dcc.Graph(id="radar"), md=6),
    #     dbc.Col(dcc.Graph(id="bars"), md=6),
    # ]),
],fluid=True)

# Define a color map for phases
PHASE_COLORS = {
    "PP": "#FFD700",  # Gold for Powerplay
    "Middle": "#1E90FF", # DodgerBlue for Middle overs
    "Death": "#DC143C" # Crimson for Death overs
}

# =============================== CALLBACK ===============================
@app.callback(
    Output("batkpi","children"),
    Output("runs","figure"),
    Output("wkts","figure"),
    Output("form","figure"),
    Output("dismiss","figure"),
    Output("batphase","figure"),
    Output("bowlphase","figure"),
    Output("animated_runs","figure"), # Added new output for animated plot
    Input("p1","value"),Input("season","value")
)
def update(p1,season):

    bat = bat_summary(p1); bowl = bowl_summary(p1)
    team = get_team_abbr(base(p1).batting_team.mode().iloc[0]) if not base(p1).empty else "UNK"
    color = get_team_color(base(p1).batting_team.mode().iloc[0]) if not base(p1).empty else "#888"

    combined_kpi = dbc.Card(
    dbc.CardBody([
        html.H4(f"{p1}  •  {team}", style={"fontWeight": "700", "color": color}),
        html.Hr(style={"borderTop": f"2px solid {color}"}),

        dbc.Row([
            dbc.Col([
                html.H5("Batting Performance", style={"color": "#3EC1D3", "fontWeight": "600"}),
                html.P([
                    html.Span(f"Runs: ", style={"fontWeight": "600"}),
                    html.Span(str(bat['r'])), html.Br(),
                    html.Span(f"Strike Rate: ", style={"fontWeight": "600"}),
                    html.Span(str(bat['sr'])), html.Br(),
                    html.Span(f"4s / 6s: ", style={"fontWeight": "600"}),
                    html.Span(f"{bat['4']} / {bat['6']}"), html.Br(),
                    html.Span(f"Boundary %: ", style={"fontWeight": "600"}),
                    html.Span(str(bat['bp'])+"%"),
                ], style={"fontSize": "15px"})
            ], md=6),

            dbc.Col([
                html.H5("Bowling Performance", style={"color": "#F76C6C", "fontWeight": "600"}),
                html.P([
                    html.Span(f"Wickets: ", style={"fontWeight": "600"}),
                    html.Span(str(bowl['w'])), html.Br(),
                    html.Span(f"Economy: ", style={"fontWeight": "600"}),
                    html.Span(str(bowl['eco'])), html.Br(),
                    html.Span(f"Bowling SR: ", style={"fontWeight": "600"}),
                    html.Span(str(bowl['sr'])), html.Br(),
                    html.Span(f"Dot %: ", style={"fontWeight": "600"}),
                    html.Span(str(bowl['dot'])+"%"),
                ], style={"fontSize": "15px"})], md=6),]),]),style={"backgroundColor": "#111111", "border": f"2px solid {color}", "borderRadius": "12px"})


    # Season-wise Runs for P1
    s_p1 = base(p1)[lambda x:x.is_bat & valid_ball(x)].groupby("season").batsman_runs.sum().reset_index()
    s_p1 = s_p1[(s_p1["season"] >= season[0]) & (s_p1["season"] <= season[1])]
    s_p1['Player'] = p1

    # Season-wise Wickets for P1
    w_p1 = base(p1)[lambda x:x.is_bowl & (x.is_wicket==1) & x.player_dismissed.notna()
                 & (~x.dismissal_kind.isin(["Run Out","Obstructing The Field","Retired Hurt","Retired Out",
                                            "Handled The Ball","Timed Out"]))].groupby("season").size().reset_index(name="wkts")
    w_p1 = w_p1[(w_p1["season"] >= season[0]) & (w_p1["season"] <= season[1])]
    w_p1['Player'] = p1

    fig_runs = px.bar(s_p1, x="season", y="batsman_runs", title="Season Runs", color="season")
    fig_wkts = px.line(w_p1, x="season", y="wkts", title="Season Wickets", markers=True)


    # Form (rolling)
    f = base(p1)[lambda x:x.is_bat & valid_ball(x)].groupby(["match_id","date"]).batsman_runs.sum().reset_index().sort_values("date")
    f["rolling"]=f.batsman_runs.rolling(5,min_periods=1).mean()
    fig_form = px.line(f,x="date",y=["batsman_runs","rolling"],title="Form Curve")

    # Dismissal Pie
    dp = base(p1)[lambda x:(x.is_wicket==1)&(x.player_dismissed==p1)]
    fig_dismiss = px.pie(dp, names="dismissal_kind", title="Dismissal Breakdown")

    # Batting Phase
    bp = base(p1)[lambda x:x.is_bat & valid_ball(x)]
    bp["phase"]=bp.over.apply(lambda o:"PP" if o<=6 else "Middle" if o<=15 else "Death")
    bp=bp.groupby("phase").batsman_runs.sum().reset_index()
    fig_bphase = px.bar(bp,x="phase",y="batsman_runs",title="Batting by Phase", color='phase', color_discrete_map=PHASE_COLORS)

    # Bowling Phase
    bl = base(p1)[lambda x:x.is_bowl & valid_ball(x)]
    bl["phase"]=bl.over.apply(lambda o:"PP" if o<=6 else "Middle" if o<=15 else "Death")
    bl=bl.groupby("phase").total_runs.mean().reset_index()
    fig_bowlphase = px.bar(bl,x="phase",y="total_runs",title="Average runs conceded per phase", color='phase', color_discrete_map=PHASE_COLORS)

    # Animated Season-wise Runs (for all seasons)
    s_all_seasons = base(p1)[lambda x:x.is_bat & valid_ball(x)].groupby("season").batsman_runs.sum().reset_index()
    # Ensure all seasons are present for animation, even if 0 runs
    all_seasons_df = pd.DataFrame({"season": SEASONS})
    s_all_seasons = pd.merge(all_seasons_df, s_all_seasons, on="season", how="left").fillna(0)

    # Dynamically adjust y-axis range
    max_runs_animated = s_all_seasons['batsman_runs'].max()
    animated_y_range = [0, max(100, max_runs_animated * 1.1)] # Ensure a minimum range

    # Dynamically adjust x-axis range based on actual data presence
    max_season_with_runs = s_all_seasons[s_all_seasons['batsman_runs'] > 0]['season'].max()
    # If player never scored runs, default to showing all seasons
    if pd.isna(max_season_with_runs):
        animated_xaxis_max = max(SEASONS) + 0.5
    else:
        animated_xaxis_max = max_season_with_runs + 0.5

    fig_animated_runs = px.bar(s_all_seasons, x="season", y="batsman_runs",
                                 animation_frame="season", animation_group="season",
                                 range_y=animated_y_range,
                                 title="Animated Season-wise Runs",
                                 labels={"batsman_runs": "Runs"})
    fig_animated_runs.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1000
    fig_animated_runs.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 500
    fig_animated_runs.update_layout(xaxis_range=[min(SEASONS) - 0.5, animated_xaxis_max])

    # Removed Comparison Radar and Bars logic
    fig_radar, fig_bar = go.Figure(), go.Figure()

    return combined_kpi, fig_runs, fig_wkts, fig_form, fig_dismiss, fig_bphase, fig_bowlphase, fig_animated_runs

# RUN
if __name__=="__main__":
    app.run(debug=True,port=8053)

<IPython.core.display.Javascript object>